In [ ]:
!pip install grpcio tensorflow-serving-api

In [2]:
!pip install keras-image-helper

In [1]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2023-12-04 16:19:22.983210: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 16:19:23.039561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 16:19:23.039621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 16:19:23.040931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 16:19:23.048218: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 16:19:23.051493: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
from keras_image_helper import create_preprocessor

In [4]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [5]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [6]:
X_pr = tf.make_tensor_proto(X, shape=X.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(X_pr)

In [8]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8500: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8500: Failed to connect to remote host: Connection refused {grpc_status:14, created_time:"2023-12-04T16:19:40.642680472+05:30"}"
>

In [26]:
preds = pb_response.outputs['dense_7'].float_val

In [27]:
classes = [
  'dress',
  'hat', 
  'longsleeve',
  'outwear',
  'pants',
  'shirt',
  'shoes',
  'shorts',
  'skirt',
  't-shirt'
]

In [31]:
dict(zip(classes, preds))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595333099365234,
 'outwear': -1.0892642736434937,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.648310422897339,
 'shorts': 3.2411563396453857,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}